In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
pd.options.display.max_rows = 100
from rich import print
import warnings
import datetime
from astropy.time import Time
import os
import astropy.units as u
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
import astropy.coordinates as coord
import astropy.units as u
import time
from gfcat_utils import parse_lightcurves_csv,parse_exposure_time,read_image
from matplotlib import gridspec
from astropy.visualization import ZScaleInterval
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
from scipy.interpolate import interp1d
import tqdm

In [2]:
def angularSeparation(ra1, dec1, ra2, dec2):
    d2r = np.pi/180.
    ra2deg = 1./d2r
    d1 = dec1*d2r,
    d2 = dec2*d2r
    r1 = ra1*d2r
    r2 = ra2*d2r
    a = np.sin((d2-d1)/2.)**2.+np.cos(d1)*np.cos(d2)*np.sin((r2-r1)/2.)**2.
    r = 2*np.arcsin(np.sqrt(a))
    return r*ra2deg

def counts2flux(cps, band):
    scale = 1.4e-15 if band == 'FUV' else 2.06e-16
    return scale*cps

def counts2mag(cps, band):
    scale = 18.82 if band == 'FUV' else 20.08
    # This threw a warning if the countrate was negative which happens when
    #  the background is brighter than the source. Suppress.
    with np.errstate(invalid='ignore'):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mag = -2.5 * np.log10(cps) + scale
    return mag

def mag2counts(mag, band):
    scale = 18.82 if band == 'FUV' else 20.08
    return 10.**(-(mag-scale)/2.5)

In [3]:
tbl = pd.read_csv('gfcat_visit_table_lcstats.csv',index_col=0)
tbl

,eclipse,obj_id,ra,dec,simbad_n_match,simbad_otype,simbad_main_id,simbad_distance,simbad_match_offset,obstart,...,skybg_flux_NUV,skybg_flux_err_NUV,expt_eff_FUV,skybg_counts_FUV,skybg_counts_err_FUV,skybg_flux_FUV,skybg_flux_err_FUV,morphology,qa_flag_nuv,qa_flag_fuv
0,927,3715900927.0,249.909813,41.112522,1,RRLyrae,V* AF Her,3149.6062992125985,2.2239209651969762,741051678,...,--,--,--,--,--,--,--,--,0,0
1,1413,1953301413.0,311.565113,-4.930670,0,Low-Mass*,2MASS J20461427-0456025,817.8621084485156,23.58975724917362,743928009,...,--,--,--,--,--,--,--,F,0,0
2,1420,1468701420.0,315.935391,-7.379244,1,Low-Mass*,2MASS J21034437-0722434,484.8719937936385,0.7203244462531776,743969866,...,--,--,--,--,--,--,--,F,0,0
3,1422,1479501422.0,343.352654,-39.793072,0,Galaxy,LEDA 2794348,--,31.206438874927272,743982262,...,--,--,--,--,--,--,--,F,0,0
4,1534,1091701534.0,259.445075,59.689607,1,Star,2MASS J17174656+5941240,129.1522446660123,2.3970117655300758,744644158,...,--,--,--,--,--,--,--,F,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,46790,--,188.868257,-36.870892,1,"RRLyrae, Star",CRTS J123528.2-365214,7473.841554559043,2.5626142878286764,1012392870,...,--,--,--,--,--,--,--,--,0,0
2060,46807,--,183.176317,-26.926904,1,HighPM*,2MASS J12124204-2655328,43.672699322199705,5.210579871203921,1012493371,...,--,--,--,--,--,--,--,--,0,0
2061,46818,--,184.069175,-30.364380,1,Star,UCAC2 19153424,1367.2409078479627,8.818929739376449,1012558403,...,--,--,--,--,--,--,--,--,0,0
2063,46828,--,183.710681,-32.431776,0,--,--,--,--,1012618574,...,--,--,--,--,--,--,--,F,0,0


In [4]:
# make final QA plots
for i in tqdm.tqdm(tbl.index.tolist()[1720:]):
    visit = tbl.loc[i]
    e = visit['eclipse']
    edir = str(e).zfill(5)
    epath = f"../data/lightcurves/e{edir}"
    ra,dec = visit['ra'],visit['dec']
    otype = visit['simbad_otype'].split(',')[0]
    obs = {}
    for band in ['NUV','FUV']:
        fn = f"{epath}/e{edir}-{band[0].lower()}d-30s-photom-17_5.csv"
        try:
            expt = parse_exposure_time(f"{epath}/e{edir}-{band[0].lower()}d-30s-exptime.csv")
            lcs = parse_lightcurves_csv(fn)
            # a few eclipse have multiple variables, so we have to pick the right one
            ix = np.argmin(angularSeparation(ra,dec,
                           np.array([lc['ra'] for lc in lcs]),
                           np.array([lc['dec'] for lc in lcs])))
            lc = lcs[ix]
        except FileNotFoundError:
            continue
        obs[band] = [lc,expt]

    if not len(obs.keys()):
        print(f"There is no lightcurve data available for {edir}")
        continue
    
    # This block is to help test whether all the data is present
    #for k in obs.keys():
    #    imgfilename = f"{epath}/e{edir}-{k[0].lower()}d-full.fits.gz"
    #    if not os.path.exists(imgfilename):
    #        print(f"Missing: {imgfilename}")
    #continue
        
    if len(obs.keys())==1: # only a single band is present
        gs = gridspec.GridSpec(ncols=10,nrows=7)
        fig = plt.figure(figsize=(10,7))
        tmin,tmax = (obs[list(obs.keys())[0]][1]['t0'][0]-15,
                     obs[list(obs.keys())[0]][1]['t0'][-1]+15)
    else:
        gs = gridspec.GridSpec(ncols=10,nrows=16,
                               #wspace=0.01,hspace=0.5,
                              )
        fig = plt.figure(figsize=(10,16))
        tmin,tmax = (min(obs['NUV'][1]['t0'][0],obs['FUV'][1]['t0'][0])-10,
                     max(obs['NUV'][1]['t0'][-1],obs['FUV'][1]['t0'][-1])+20)
    #fig.tight_layout()
    
    for j,k in enumerate(obs.keys()):
        c = 'b' if k=='FUV' else 'r' # set the graphic color for the band
        ra,dec = obs[k][0]['ra'],obs[k][0]['dec']

        imgfilename = f"{epath}/e{edir}-{k[0].lower()}d-full.fits.gz"
        imgmap, _, _, wcs, tranges, exptimes = read_image(imgfilename)
        imgmap[np.where(np.isinf(imgmap))] = 0  # because it pops out with inf values... IDK
        imgmap[np.where(imgmap < 0)] = 0
        
        imgpos = wcs.wcs_world2pix([[lc['ra'],lc['dec']]],1) # set the origin to FITS standard
        imgx,imgy = imgpos[0]
        imsz = np.shape(imgmap)[-2:]
        
        # crop on the subframe
        # noting that image coordinates and numpy coordinates are flipped
        boxsz = 150
        x1, x2, y1, y2 = (max(int(imgy - boxsz), 0),
                          min(int(imgy + boxsz), imsz[0]),
                          max(int(imgx - boxsz), 0),
                          min(int(imgx + boxsz), imsz[1]))

        # Full frame
        offset=0 if j==0 else 5
        ax = fig.add_subplot(gs[offset:5+offset,:5],projection=wcs)
        ##ax.imshow(ZScaleInterval()(imgmap[x1_:x2_, y1_:y2_]), origin="lower", cmap="Greys_r")
        ax.imshow(ZScaleInterval()(imgmap), origin="lower", cmap="Greys_r")
        ax.set_xticks([])
        ax.set_yticks([])
        x1_,y1_=0,0
        rect = Rectangle((y1 - y1_, x1 - x1_), 2 * boxsz, 2 * boxsz,
                         linewidth=1, edgecolor='y', facecolor='none',ls='solid')
        ax.add_patch(rect)
        circ = Circle((y1+boxsz,x1+boxsz), 17.5/1.5,
            linewidth=1, edgecolor=c, facecolor='none', ls='solid')
        ax.add_patch(circ)
        circ = Circle((boxsz, boxsz), 25/1.5,
            linewidth=2, edgecolor=c, facecolor='none', ls='dotted')
        ax.add_patch(circ)
        circ = Circle((boxsz, boxsz), 35/1.5,
            linewidth=2, edgecolor=c, facecolor='none', ls='dotted')
        ax.add_patch(circ)
        ax.text(25, 62, k, color=c, fontsize=15,
           bbox={'facecolor': 'black', 'alpha': 1, 'pad': 2})
        ax.coords[0].set_axislabel('galactic longitude')#, minpad=0.3)
        ax.coords[1].set_axislabel('galactic latitude')#, minpad=-0.4)
        ax.coords[0].set_major_formatter('dd:mm')#:ss.s')
        ax.coords[1].set_major_formatter('dd:mm')
        ax.coords[0].set_ticks(number=6)
        ax.coords[1].set_ticks(number=6)
        ax.coords[0].set_ticks_position('' if j else 't')
        ax.coords[0].set_ticklabel_position('' if j else 't')
        ax.coords[0].set_axislabel_position('' if j else 't')
        ax.coords[1].set_ticks_position('l')
        ax.coords[1].set_ticklabel_position('l')
        ax.coords[1].set_axislabel_position('l')
        ax.coords[0].tick_params(labelsize=8)
        ax.coords[1].tick_params(labelsize=8)
        #ax.xaxis.label.set_fontsize(6)
        #ax.yaxis.label.set_fontsize(6)

        # Thumbnail
        ax = fig.add_subplot(gs[offset:5+offset,5:],projection=wcs)
        ax.imshow(ZScaleInterval()(imgmap[x1:x2, y1:y2]), origin="lower", cmap="Greys_r")
        ax.set_xticks([])
        ax.set_yticks([])
        rect = Rectangle((0,0), 2 * boxsz, 2 * boxsz,
                         linewidth=4, edgecolor='y', facecolor='none',ls='solid')
        ax.add_patch(rect)

        circ = Circle((boxsz, boxsz), 17.5/1.5,
                      linewidth=2, edgecolor=c, facecolor='none', ls='solid')
        ax.add_patch(circ)
        circ = Circle((boxsz, boxsz), 25/1.5,
                      linewidth=2, edgecolor=c, facecolor='none', ls='dashed')
        ax.add_patch(circ)
        circ = Circle((boxsz, boxsz), 35/1.5,
                      linewidth=2, edgecolor=c, facecolor='none', ls='dashed') 
        #ax.text(0, 62, otype, color=c, fontsize=15,
        #   bbox={'facecolor': 'black', 'alpha': 1, 'pad': 2})
        ax.add_patch(circ)
        ax.coords[0].set_axislabel('galactic longitude')#, minpad=0.3)
        ax.coords[1].set_axislabel('galactic latitude')#, minpad=-0.4)
        ax.coords[0].set_major_formatter('dd:mm')#:ss.s')
        ax.coords[1].set_major_formatter('dd:mm')
        ax.coords[0].set_ticks(number=6)
        ax.coords[1].set_ticks(number=6)
        ax.coords[0].set_ticks_position('' if j else 't')
        ax.coords[0].set_ticklabel_position('' if j else 't')
        ax.coords[0].set_axislabel_position('' if j else 't')
        ax.coords[1].set_ticks_position('r')
        ax.coords[1].set_ticklabel_position('r')
        ax.coords[1].set_axislabel_position('')
        ax.coords[0].tick_params(labelsize=8)
        ax.coords[1].tick_params(labelsize=8)


        # Lightcurve
        offset = 10 if len(obs.keys())==2 else 5 # for correct positioning in the gridspec
        ax = fig.add_subplot(gs[offset+j*2:offset+2+j*2,:])
        
        # There appears to be ramp-up phenomenon in FUV in this eclipse range
        lims = (3,None) if ((k=='FUV') and (e>=12889) and (e<=13856)) else (0,None)

        t = np.array(obs[k][1]['t0'])[lims[0]:lims[1]]
        flux = counts2flux(obs[k][0]['cps'],k)[lims[0]:lims[1]]*10**15
        flux_err = counts2flux(obs[k][0]['cps_err'],k)[lims[0]:lims[1]]*10**15
        tix = np.where(np.isfinite(flux))
        X_=np.linspace(t[tix].min(), t[tix].max(), 500)
        interp_model = interp1d(t[tix],
                flux[tix]+3*flux_err[tix],
                kind = "slinear")
        Y_up=interp_model(X_)
        interp_model = interp1d(t[tix],
                flux[tix]-3*flux_err[tix],
                kind = "slinear")
        Y_down=interp_model(X_)

        ax.fill_between(X_,
                Y_up,Y_down,
                alpha=0.1,facecolor=c,edgecolor=c)

        ax.errorbar(obs[k][1]['t0'][lims[0]:lims[1]],
                    counts2flux(obs[k][0]['cps'],k)[lims[0]:lims[1]]*10**15,
                    yerr=1*counts2flux(obs[k][0]['cps_err'],k)[lims[0]:lims[1]]*10**15,
                    fmt=f'none',
                    color=c,label=k,alpha=0.4)
        #ax.errorbar(obs[k][1]['t0'],counts2flux(obs[k][0]['cps'],k)*10**15,
        #            yerr=0*counts2flux(obs[k][0]['cps_err'],k)*10**15,
        #            fmt=f'{c}-',label=k)
        # Plot the curve line of data.
        interp_model = interp1d(t[tix],
                        flux[tix],
                        kind = "slinear")
        X_=np.linspace(t[tix].min(), t[tix].max(), 500)
        Y_=interp_model(X_)
        plt.plot(X_,Y_,f'{c}-')
        
        # draw a line for the background flux
        #plt.hlines([tbl.iloc[i][f'skybg_flux_{k}']],
        #           tmin,tmax,
        #           linestyle='dashed',color='k',alpha=0.2)
        
        ax.set_xlim([tmin,tmax])
        ax.set_ylabel(f'{k} Flux\n'+r'$femtoerg·sec^{-1}cm^{-2}Å^{-1}$')
        # show the time-axis here only if this is a single-band observation
        if len(obs.keys())==1:
            ticks = np.arange(obs[k][1]['t0'][lims[0]:lims[1]][0]-15,
                                    obs[k][1]['t0'][lims[0]:lims[1]][-1]-15,100)
            ax.set_xticks(ticks)
            ax.set_xticklabels(list(np.arange(0,len(ticks))*100))
            ax.set_xlabel('seconds from start of observation')
        else:
            ax.set_xticks([])
        
        ax.yaxis.set_major_formatter(FormatStrFormatter('%0.2f'))
        ax.yaxis.set_major_locator(MaxNLocator(3))
        ax.xaxis.label.set_fontsize(8)
        ax.yaxis.label.set_fontsize(8)
        ax.tick_params(labelsize=8)
        ax.hlines([np.nanmin(counts2flux(obs[k][0]['cps'],k)[lims[0]:lims[1]]*10**15),
                   np.nanmax(counts2flux(obs[k][0]['cps'],k)[lims[0]:lims[1]]*10**15)],
                  tmin,tmax,
                  linestyle='dotted',color='k',alpha=0.5)

        # put the mean ab magnitude on the right side of the plot
        ax_ = ax.twinx()
        ax_.tick_params(labelsize=8)
        ax_.set_ylim(ax.get_ybound())
        if np.nanmin(obs[k][0]['cps'][lims[0]:lims[1]])!=0:
            ax_.set_yticks([
                np.nanmin(counts2flux(obs[k][0]['cps'],k)[lims[0]:lims[1]]*10**15),
                np.nanmax(counts2flux(obs[k][0]['cps'],k)[lims[0]:lims[1]]*10**15)])
            ax_.set_yticklabels(
                [f"{np.round(counts2mag(np.nanmin(obs[k][0]['cps'][lims[0]:lims[1]]),k),2)}\nAB Mag",
                 f"{k}\n{np.round(counts2mag(np.nanmax(obs[k][0]['cps'][lims[0]:lims[1]]),k),2)}"])
        else: # the minimum in this band is actually zero, which has no valid mag
            ax_.set_yticks([np.nanmax(counts2flux(obs[k][0]['cps'],k)[lims[0]:lims[1]]*10**15)])
            ax_.set_yticklabels([
                f"{k}\n{np.round(counts2mag(np.nanmax(obs[k][0]['cps'][lims[0]:lims[1]]),k),2)}\nAB Mag"])
    
    # the ratio plots
    if len(obs.keys())==2:
        ax = fig.add_subplot(gs[14:,:])
        _,ix,_ = np.intersect1d(obs['NUV'][1]['t0'],obs['FUV'][1]['t0'],return_indices=True)
        if not len(ix):
            print(f"Time bins do not overlap for e{e}")
            continue
        # we chop the last bin because a low exposure last bin can dominate the plot
        color = (counts2flux(obs['FUV'][0]['cps'],'FUV')[ix][lims[0]:-1] /
                 counts2flux(obs['NUV'][0]['cps'],'NUV')[ix][lims[0]:-1])
        color_err = color * np.sqrt(
            (counts2flux(obs['NUV'][0]['cps_err'],'NUV')[ix][lims[0]:-1] /
             counts2flux(obs['NUV'][0]['cps'],    'NUV')[ix][lims[0]:-1])**2 +
            (counts2flux(obs['FUV'][0]['cps_err'],'FUV')[ix][lims[0]:-1] /
             counts2flux(obs['FUV'][0]['cps'],    'FUV')[ix][lims[0]:-1])**2)
        t = (np.array(obs['NUV'][1]['t0'])[ix][lims[0]:-1] -
             np.nanmin(np.array(obs['NUV'][1]['t0'])[ix][lims[0]:-1])+15+30*lims[0])
        tix = np.where(np.isfinite(color) & np.isfinite(color_err))
        X_=np.linspace(t[tix].min(), t[tix].max(), 500)

        # Plot the curved envelope
        interp_model = interp1d(t[tix],
                        color[tix]+3*color_err[tix],
                        kind = "slinear")
        Y_up=interp_model(X_)
        interp_model = interp1d(t[tix],
                color[tix]-3*color_err[tix],
                kind = "slinear")
        Y_down=interp_model(X_)

        ax.fill_between(X_,
                Y_up,Y_down,
                alpha=0.1,facecolor='k',edgecolor='k')

        ax.errorbar(t,
                    color,yerr=1*color_err,fmt='none',color='k',alpha=0.3)

        # Plot the curve line of data.
        interp_model = interp1d(t[tix],
                        color[tix],
                        kind = "slinear")
        X_=np.linspace(t[tix].min(), t[tix].max(), 500)
        Y_=interp_model(X_)
        plt.plot(X_,Y_,'k-')
        
        ax.set_xlim([tmin-tmin,tmax-tmin])
        ax.set_xlabel('seconds from start of observation')
        ax.set_ylabel('FUV / NUV\nFlux Ratio')
        ax.yaxis.set_major_formatter(FormatStrFormatter('%0.2f'))
        ax.yaxis.set_major_locator(MaxNLocator(3))
        ax.yaxis.label.set_fontsize(8)
        ax.xaxis.label.set_fontsize(8)
        ax.yaxis.label.set_fontsize(8)
        ax.tick_params(labelsize=8)
    
    plot_fn=f'../data/final_qa_plots/gfcat_{edir}_{str(i).zfill(4)}.jpg'
    plt.savefig(plot_fn,bbox_inches='tight', pad_inches=0.1, dpi=500)
    plt.close('all')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 239/239 [19:13<00:00,  4.82s/it]


In [6]:
for i in np.arange(len(tbl)):
    nbands = sum([tbl.iloc[i]['mag_mean_NUV']!='--',tbl.iloc[i]['mag_mean_FUV']!='--'])
    if nbands==1:
        whichband = 'FUV' if tbl.iloc[i]['mag_mean_NUV']=='--' else 'NUV'
    # Auto generate a caption for the QA image from the table data.
    c1 = f"""This object at RA={np.around(tbl.iloc[i]['ra'],5)}°, Dec={np.round(tbl.iloc[i]['dec'],5)}°
was cleanly observed in {'both bands' if nbands==2 else whichband} on {tbl.iloc[i]['datetime_iso'][:-9]} during eclipse {tbl.iloc[i]['eclipse']}.""".replace('\n',' ')
    morphkey = {'T':'trending','F':'flaring','E':'eclipsing','U':'unknown'}
    c4 = f"""It was tagged as type \"{morphkey[tbl.iloc[i]['morphology']]}\" during manual review, based only on the single-visit lightcurve morphology."""
    if tbl.iloc[i]['simbad_otype']=='--':
        c2 = f"""There is not a good match to any source in SIMBAD."""
    else:
        c2 = f"""Likely matching sources in SIMBAD with known type have type {tbl.iloc[i]['simbad_otype'].replace(',',' or')}."""
    if tbl.iloc[i]['simbad_n_match']==0 and tbl.iloc[i]['gaia_n_match']==0:
        c2 = ''
        c3 = f"""There are no matching sources within 12.5" of the GALEX position in either SIMBAD or Gaia."""
        c5 = f"""The nearest matching Gaia source is at a distance of {np.round(float(tbl.iloc[i]['distance'])/1000,3)}"""
    else:
        c3 = f"""The best matched distance is {np.round(float(tbl.iloc[i]['distance'])/1000,3)} kpc."""
        c5 = f"""There are {tbl.iloc[i]['simbad_n_match']} matching SIMBAD sources and {tbl.iloc[i]['gaia_n_match']} Gaia sources within 12.5" of the GALEX source position."""
    autocaption = str(' ').join([c1,c4,c2,c3,c5]).replace('  ',' ')
    print(autocaption)

NameError: name 'whichband' is not defined